In [1]:
# I want to create tables for each season

#Consiter these table ideas
# player info
# passing info
# rushing info
# receiving info
# kicking info
# punting info
# offense info
# defense info


In [2]:
import pandas as pd

import sqlalchemy as sal
from sqlalchemy import create_engine
from sqlalchemy.sql import text

import streamlit as st

from datetime import datetime
import time

In [3]:
team_names=[
    '49ers',
    'Bears',
    'Bengals',
    'Broncos',
    'Browns',
    'Buccaneers',
    'Buffalos',
    'Cardinals',
    'Chargers',
    'Chiefs',
    'Colts',
    'Commanders',
    'Cowboys',
    'Dolphins',
    'Eagles',
    'Falcons',
    'Giants',
    'Jaguars',
    'Jets',
    'Lions',
    'Packers',
    'Panthers',
    'Patriots',
    'Raiders',
    'Rams',
    'Ravens',
    'Saints',
    'Seahawks',
    'Steelers',
    'Texans',
    'Titans',
    'Vikings'
]

In [4]:
years = range(2010,2024)

In [5]:
table_name_li = [
    'team_stats_and_ranking',
    'schedule_and_game_results',
    'team_conversions',
    'passing',
    'rushing_and_receiving',
    'kick_and_punt_returns',
    'kicking',
    'punting',
    'defense_and_fumbles',
    'scoring_summary',
    'touchdown_log',
    'opponent_touchdown_log'
]

In [6]:
# connection link varaibles
# NFL Data databas has all year and team in separate tables
servername = '*****'
nfl_data = 'NFL_Data'
trusted_conneciton = '?trusted_conneciton=yes'
driver = '&driver=ODBC+Driver+17+for+SQL+Server'

In [7]:
#create engine url to connect to sql server
engine_nfl_data = create_engine(f'mssql+pyodbc://@{servername}/{nfl_data}{trusted_conneciton}{driver}')
engine_nfl_data

Engine(mssql+pyodbc://@DESKTOP-5IAPFQC/NFL_Data?driver=ODBC+Driver+17+for+SQL+Server&trusted_conneciton=yes)

In [8]:
#Conneciton to SQL Engine
conn_nfl_data = engine_nfl_data.connect()

In [9]:
#connect overall tables
servername = '*****'
football_data = 'Football_Data'
trusted_conneciton = '?trusted_conneciton=yes'
driver = '&driver=ODBC+Driver+17+for+SQL+Server'

In [10]:
engine_football_data = create_engine(f'mssql+pyodbc://@{servername}/{football_data}{trusted_conneciton}{driver}')
engine_football_data

Engine(mssql+pyodbc://@DESKTOP-5IAPFQC/Football_Data?driver=ODBC+Driver+17+for+SQL+Server&trusted_conneciton=yes)

In [11]:
conn_football_data = engine_football_data.connect()

######import roster to sql

for year in years:
    for team in team_names:
        file_loc = f'NFL_Data_{year}/{team}/{team}_roster'
            
        df = pd.read_csv(file_loc)

        df.to_sql(f'Roster_{team}_{year}', con=conn_nfl_data, if_exists='replace')

# roster to sql union all teams per season
for year in years:
    #database
    football_db = f'Football_Data_{year}'

    #db connection str
    football_data_eng = create_engine(f'mssql+pyodbc://@{servername}/{football_db}{trusted_conneciton}{driver}')

    football_data_year_conn = football_data_eng.connect()


    players_li = []
    
    for team in team_names:
        players = f'select * from Roster_{team}_{year}'

        players_li.append(players)

    
    players_union = ' union '.join([p for p in players_li])

        
    df = pd.read_sql(players_union, con=conn_nfl_data, index_col='index')

    df.to_sql(f'Players', con=football_data_year_conn, if_exists='replace')

#import tables to each season by union from 
#nfl_data (all individual datatables per team an season)
for table in table_name_li:
    for year in years:
        #database
        football_db = f'Football_Data_{year}'
    
        #db connection str
        football_data_eng = create_engine(f'mssql+pyodbc://@{servername}/{football_db}{trusted_conneciton}{driver}')
    
        football_data_year_conn = football_data_eng.connect()
    
    
        union_table_li = []
    
        for team in team_names:
            query_str = f'select * from {table}_{team}_{year}'
        
            union_table_li.append(query_str)
        
    
        print(year, table)
    
        union_table_query_str = ' union '.join([p for p in union_table_li])
        
        df = pd.read_sql(union_table_query_str, con=conn_nfl_data)
    
        df.to_sql(f'{table}', con=football_data_year_conn, if_exists='replace')

# Team Tendencies

In [12]:
#Play tendencies column meaning
#table-nfl-team-tendencies_year

##pass rate - passing plays %
##rush rate - rushing plays %
##proe - pass rate over expectation
##personnel_11,12,13,21,22, other - number of running backs (1,2) and tightends(1,2) 
##form_shotgun - shotgun formation
##form_single - single wing formation
##form_iform - qb, fb, hb behind center
##form_empty - 
##form_other
##adot - how far the ball was thrown and received
##to_sticks - 
##pass_left, middle, right - pass direction
##wr_target - targeted wide receiver to move the ball forward
##te_target - targeted tight end to move the ball forward
##rb_target - targeted running back to move the ball forward
##run_left, right, guard, tackle, end - run direction
##go_fourth - go on fourthdown
##go_two - go for 2 pt conversion
##rec_gini - 
##rush_gini - 
##personnel_gini - 
##formation_gini - 
##avg_top - 

In [13]:
df = pd.read_csv('Team Play Tendencies/nfl-team-tendencies_2016', index_col=0)

In [14]:
df

Unique_Id  Season Team   Team_Nick              Team_Name  \
0   ARI_2016    2016  ARI   Cardinals      Arizona Cardinals   
1   ATL_2016    2016  ATL     Falcons        Atlanta Falcons   
2   BAL_2016    2016  BAL      Ravens       Baltimore Ravens   
3   BUF_2016    2016  BUF       Bills          Buffalo Bills   
4   CAR_2016    2016  CAR    Panthers      Carolina Panthers   
5   CHI_2016    2016  CHI       Bears          Chicago Bears   
6   CIN_2016    2016  CIN     Bengals     Cincinnati Bengals   
7   CLE_2016    2016  CLE      Browns       Cleveland Browns   
8   DAL_2016    2016  DAL     Cowboys         Dallas Cowboys   
9   DEN_2016    2016  DEN     Broncos         Denver Broncos   
10  DET_2016    2016  DET       Lions          Detroit Lions   
11   GB_2016    2016   GB     Packers      Green Bay Packers   
12  HOU_2016    2016  HOU      Texans         Houston Texans   
13  IND_2016    2016  IND       Colts     Indianapolis Colts   
14  JAX_2016    2016  JAX     Jaguars   Jacksonville Jaguars   
15   KC_2016    2016   KC      Chiefs     Kansas City Chiefs   
16  LAR_2016    2016  LAR        Rams       Los Angeles Rams   
17  LAC_2016    2016  LAC    Chargers   Los Angeles Chargers   
18  OAK_2016    2016  OAK     Raiders      Las Vegas Raiders   
19  MIA_2016    2016  MIA    Dolphins         Miami Dolphins   
20  MIN_2016    2016  MIN     Vikings      Minnesota Vikings   
21   NE_2016    2016   NE    Patriots   New England Patriots   
22   NO_2016    2016   NO      Saints     New Orleans Saints   
23  NYG_2016    2016  NYG      Giants        New York Giants   
24  NYJ_2016    2016  NYJ        Jets          New York Jets   
25  PHI_2016    2016  PHI      Eagles    Philadelphia Eagles   
26  PIT_2016    2016  PIT    Steelers    Pittsburgh Steelers   
27  SEA_2016    2016  SEA    Seahawks       Seattle Seahawks   
28   SF_2016    2016   SF       49ers    San Francisco 49ers   
29   TB_2016    2016   TB  Buccaneers   Tampa Bay Buccaneers   
30  TEN_2016    2016  TEN      Titans       Tennessee Titans   
31  WAS_2016    2016  WAS  Commanders  Washington Commanders   

                                       Team_Logo_Espn Team_Color  Pass_Rate  \
0   https://a.espncdn.com/i/teamlogos/nfl/500/ari.png    #97233F   0.640741   
1   https://a.espncdn.com/i/teamlogos/nfl/500/atl.png    #A71930   0.601215   
2   https://a.espncdn.com/i/teamlogos/nfl/500/bal.png    #241773   0.674812   
3   https://a.espncdn.com/i/teamlogos/nfl/500/buf.png    #00338D   0.566468   
4   https://a.espncdn.com/i/teamlogos/nfl/500-dark...    #0085CA   0.584366   
5   https://a.espncdn.com/i/teamlogos/nfl/500/chi.png    #0B162A   0.615866   
6   https://a.espncdn.com/i/teamlogos/nfl/500/cin.png    #FB4F14   0.600577   
7   https://a.espncdn.com/i/teamlogos/nfl/500/cle.png    #FF3C00   0.671414   
8   https://a.espncdn.com/i/teamlogos/nfl/500/dal.png    #002244   0.542254   
9   https://a.espncdn.com/i/teamlogos/nfl/500/den.png    #002244   0.625621   
10  https://a.espncdn.com/i/teamlogos/nfl/500/det.png    #0076B6   0.673868   
11   https://a.espncdn.com/i/teamlogos/nfl/500/gb.png    #203731   0.689042   
12  https://a.espncdn.com/i/teamlogos/nfl/500/hou.png    #03202F   0.597156   
13  https://a.espncdn.com/i/teamlogos/nfl/500/ind.png    #002C5F   0.657171   
14  https://a.espncdn.com/i/teamlogos/nfl/500/jax.png    #006778   0.664122   
15   https://a.espncdn.com/i/teamlogos/nfl/500/kc.png    #E31837   0.617920   
16  https://a.espncdn.com/i/teamlogos/nfl/500/lar.png    #003594   0.626316   
17  https://a.espncdn.com/i/teamlogos/nfl/500/lac.png    #007BC7   0.616832   
18   https://a.espncdn.com/i/teamlogos/nfl/500/lv.png    #000000   0.614714   
19  https://a.espncdn.com/i/teamlogos/nfl/500/mia.png    #008E97   0.573951   
20  https://a.espncdn.com/i/teamlogos/nfl/500/min.png    #4F2683   0.638833   
21   https://a.espncdn.com/i/teamlogos/nfl/500/ne.png    #002244   0.572806   
22   https://a.espncdn.com/i/teamlogos/nfl/500/no.png    #D3BC8D   0.6406

In [15]:
df.columns

Index(['Unique_Id', 'Season', 'Team', 'Team_Nick', 'Team_Name',
       'Team_Logo_Espn', 'Team_Color', 'Pass_Rate', 'Rush_Rate', 'PROE',
       'Personnel_11', 'Personnel_12', 'Personnel_13', 'Personnel_21',
       'Personnel_22', 'Personnel_Other', 'Form_Shotgun', 'Form_Single',
       'Form_Iform', 'Form_Empty', 'Form_Other', 'ADOT', 'To_Sticks',
       'Pass_Left', 'Pass_Middle', 'Pass_Right', 'Wr_Target', 'Te_Target',
       'Rb_Target', 'Run_Left', 'Run_Right', 'Run_Guard', 'Run_Tackle',
       'Run_End', 'Go_Fourth', 'Go_Two', 'Rec_Gini', 'Rush_Gini',
       'Personnel_Gini', 'Formation_Gini', 'Play_Clock_Used',
       'Plays_Per_Minute', 'Avg_Top', 'No_Huddle'],
      dtype='object')

#import tendencies to sql year 2016-2023
for year in range(2016,2024):
    football_data = f'Football_Data_{year}'
    
    engine_football_data_year = create_engine(f'mssql+pyodbc://@{servername}/{football_data}{trusted_conneciton}{driver}')

    conn_football_data_year = engine_football_data_year.connect()

    
    file_loc = f'Team Play Tendencies/nfl-team-tendencies_{year}'
    
    df = pd.read_csv(file_loc, index_col=0)

    df.to_sql('Play_Tendencies', con=conn_football_data_year, if_exists='replace')

# Play by Play

In [16]:
df = pd.read_csv('Team Play Tendencies/pbp-2014.csv',index_col=0)

In [17]:
df.columns

Index(['Game_ID', 'Game_Date', 'Quarter', 'Minute', 'Second', 'Offense_Team',
       'Defense_Team', 'Down', 'ToGo', 'Yard_Line', 'Series_First_Down',
       'Next_Score', 'Description', 'Team_Win', 'Season_Year', 'Yards',
       'Formation', 'Play_Type', 'Is_Rush', 'Is_Pass', 'Is_Incomplete',
       'Is_Touchdown', 'Pass_Type', 'Is_Sack', 'Is_Challenge',
       'Is_Challenge_Reversed', 'Challenger', 'Is_Measurement',
       'Is_Interception', 'Is_Fumble', 'Is_Penalty', 'Is_Two_Point_Conversion',
       'Is_Two_Point_Conversion_Successful', 'Rush_Direction',
       'Yard_Line_Fixed', 'Yard_Line_Direction', 'Is_Penalty_Accepted',
       'Penalty_Team', 'Is_Not_Play', 'Penalty_Type', 'Penalty_Yards'],
      dtype='object')

#import play by play to sql
for year in range(2014,2024):
    football_data = f'Football_Data_{year}'
    
    engine_football_data_year = create_engine(f'mssql+pyodbc://@{servername}/{football_data}{trusted_conneciton}{driver}')

    conn_football_data_year = engine_football_data_year.connect()
    
    file_loc = f'Team Play Tendencies/pbp-{year}.csv'

    df = pd.read_csv(file_loc, index_col=0)

    df.to_sql('Play_by_Play', con=conn_football_data_year, if_exists='replace')
    
    